In [0]:
%run ./configurations/paths_config

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS formula1_silver;
USE formula1_silver;

In [0]:
from pyspark.sql.types import StringType,IntegerType,DoubleType,TimestampType,StructType,StructField
from pyspark.sql.functions import col
from pyspark.sql import functions as sf

In [0]:
new_schema = StructType(fields=[StructField("circuitId",IntegerType(),False),
                            StructField("circuitRef",StringType(),True),
                            StructField("name",StringType(),True),
                            StructField("location",StringType(),True),
                            StructField("country",StringType(),True),
                            StructField("lat",DoubleType(),True),
                            StructField("lng",DoubleType(),True),
                            StructField("alt",IntegerType(),True),
                            StructField("url",StringType(),True)])

In [0]:
raw_df = spark.read.option("header","true").schema(new_schema).csv(raw_path+"/circuits.csv")

In [0]:
trimmed_df = raw_df.drop("url")

In [0]:
renamed_df = trimmed_df.withColumnRenamed("circuitId","circuit_id").withColumnRenamed("circuitRef","circuit_ref").withColumnRenamed("lat","latitude").withColumnRenamed("lng","longitude").withColumnRenamed("alt","altitude")

In [0]:
audited_df = renamed_df.withColumn("ingestion_time",sf.current_timestamp())

In [0]:
%sql 
SHOW CURRENT DATABASE;

In [0]:
audited_df.write.format("delta").option("mode",True).saveAsTable("vsarthicat.formula1_silver.circuits")

In [0]:
%sql
SELECT * FROM circuits;